In [8]:
%cd ..
!rm -rf epigraphnet

/content


In [9]:
!git clone https://github.com/0nur0duncu/epigraphnet.git

Cloning into 'epigraphnet'...
remote: Enumerating objects: 696, done.
remote: Counting objects: 100% (696/696), done.
remote: Compressing objects: 100% (639/639), done.
remote: Total 696 (delta 97), reused 652 (delta 53), pack-reused 0 (from 0)
Receiving objects: 100% (696/696), 3.92 MiB | 2.55 MiB/s, done.
Resolving deltas: 100% (97/97), done.


In [7]:
!git pull

remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 19 (delta 6), reused 17 (delta 4), pack-reused 0 (from 0)
Unpacking objects: 100% (19/19), 18.51 KiB | 1.16 MiB/s, done.
From https://github.com/0nur0duncu/epigraphnet
   29f2cb9..629f852  main       -> origin/main
Updating 29f2cb9..629f852
error: The following untracked working tree files would be overwritten by merge:
	data/__pycache__/__init__.cpython-312.pyc
	data/__pycache__/dataset.cpython-312.pyc
	data/__pycache__/preprocessing.cpython-312.pyc
	utils/__pycache__/__init__.cpython-312.pyc
	utils/__pycache__/metrics.cpython-312.pyc
	utils/__pycache__/training.cpython-312.pyc
Please move or remove them before you merge.
Aborting


In [10]:
%cd epigraphnet

/content/epigraphnet


In [3]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.1 MB/s eta 0:00:00


In [55]:
!rm -rf checkpoints/

In [11]:
!python run_all_experiments.py --num-runs 5


######################################################################
# Sınıflandırma Türü: İkili (Binary)
######################################################################
Veri yüklendi: 500 örnek, 2 sınıf

Model: CNN-LSTM
    Early stopping at epoch 26 (best epoch: 11, best val_acc: 100.00%)                              
    Run 1 completed - Test Acc: 94.00%, F1: 90.96%                                                  
    Early stopping at epoch 28 (best epoch: 13, best val_acc: 98.00%)                               
    Run 2 completed - Test Acc: 92.00%, F1: 88.34%                                                  
    Early stopping at epoch 24 (best epoch: 9, best val_acc: 98.00%)                                
    Run 3 completed - Test Acc: 92.00%, F1: 85.12%                                                  
    Early stopping at epoch 40 (best epoch: 25, best val_acc: 100.00%)                              
    Run 4 completed - Test Acc: 98.00%, F1: 96.75%            